<a href="https://colab.research.google.com/github/beimnet777/coffee_disease_image_segmentation/blob/main/Segmentation_with_unet_backbone2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import os
from matplotlib import pyplot as plt

In [ ]:
import cv2

In [ ]:
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
!pip install -U -q segmentation-models

In [ ]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
def change_mask_to_bw(path):
  img = cv2.imread(path)
  gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  (thresh, im_bw) = cv2.threshold(gray_image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
  return im_bw

In [ ]:
def prepare_dataset(width,height,X_path,Y_path=None):
  for (root ,dirs,files) in os.walk(X_path):

    X=np.zeros((len(files),height,width,3),dtype=np.float32)
    y=np.zeros((len(files),height,width,1),dtype=np.float32)

    index=0
    for file in files:
      image_num=file.split(".")[0]
      image_y_name=image_num+"_mask.png"
      path_x=os.path.join(X_path,file)

      if Y_path:
        path_y=os.path.join(Y_path,image_y_name)
        img_y=change_mask_to_bw(path_y)
        img_y=cv2.resize(img_y, (width, height))
        img_y=np.expand_dims(img_y,axis=-1)
        y[index]=img_y.astype(np.float32)


      img_x=cv2.imread(path_x)
      img_x=cv2.resize(img_x, (width, height))
      X[index]=img_x.astype(np.float32)

      index+=1
  
  return X,y

In [ ]:
train=prepare_dataset(512,256,"/content/drive/MyDrive/images/train","/content/drive/MyDrive/annotations/train")
test=prepare_dataset(512,256,"/content/drive/MyDrive/images/test","/content/drive/MyDrive/annotations/test")
val=prepare_dataset(512,256,"/content/drive/MyDrive/images/val","/content/drive/MyDrive/annotations/val")
     

In [ ]:
# define model
model = sm.Unet("resnet34", encoder_weights='imagenet',input_shape=(256,512,3))
model.compile('Adam', loss=sm.losses.bce_jaccard_loss, metrics=[sm.metrics.iou_score])
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data (InputLayer)              [(None, 256, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 bn_data (BatchNormalization)   (None, 256, 512, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_102 (ZeroPaddin  (None, 262, 518, 3)  0          ['bn_data[0][0]']                
 g2D)                                                                                             
                                                                                            

In [ ]:
preprocess_input1 = sm.get_preprocessing("resnet34")


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_data_generator = ImageDataGenerator(rotation_range=90,
                     width_shift_range=0.2,
                     height_shift_range=0.2,
                     shear_range=0.35,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='nearest',
                      brightness_range=(0.5,0.9),)
mask_data_generator =ImageDataGenerator(
                      rotation_range=90,
                     width_shift_range=0.2,
                     height_shift_range=0.2,
                     shear_range=0.35,
                     zoom_range=0.2,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='nearest',
                     brightness_range=(0.5,0.9)
                    )

In [ ]:
X_train=preprocess_input1(train[0])
y_train=train[1]

In [ ]:
seed=24
# image_data_generator.fit(X_train, augment=True, seed=seed)
image_generator = image_data_generator.flow(train[0], seed=seed)

# mask_data_generator.fit(y_train, augment=True, seed=seed)
mask_generator = mask_data_generator.flow(train[1], seed=seed)


In [ ]:
valid_img_generator = image_data_generator.flow(test[0], seed=seed)
valid_mask_generator = mask_data_generator.flow(test[1], seed=seed)

In [ ]:
def my_image_mask_generator(image_generator, mask_generator):
    train_generator = zip(image_generator, mask_generator)

    for (img, mask) in train_generator:
        yield (img, mask)

my_generator = my_image_mask_generator(image_generator, mask_generator)

In [ ]:
val_generator=my_image_mask_generator(valid_img_generator,valid_mask_generator)

In [ ]:
history = model.fit(my_generator, steps_per_epoch=25, epochs=5)

Epoch 1/5
 4/25 [===>..........................] - ETA: 31s - loss: -147.7880 - iou_score: 8.0492

KeyboardInterrupt: ignored

In [ ]:
model.save("leaf_segmentation.h5")

In [ ]:
for index,img in enumerate(prediction):
  img=(img>=0.5)
  img = img.astype(int)
  img*=255
  cv2_imshow(val[0][index])
  cv2_imshow(img)

In [ ]:
def load_model(path):
  model=tf.keras.models.load_model(path,compile=False)
  return model

In [ ]:
model=load_model("/content/drive/MyDrive/leaf_segmentation.h5")

In [ ]:
val2=[]
for root,dir,files in os.walk("/content/drive/MyDrive/coffee_test_annotations/coffee_test_annotations/"):
  for file in files:
    if file.split(".")[1]=="jpg":
      path=os.path.join("/content/drive/MyDrive/coffee_test_annotations/coffee_test_annotations/",file)

      img=cv2.imread(path)
      img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
      img=cv2.resize(img, (512, 256))
      
      val2.append(img)
val2=np.array(val2)
validation_2=preprocess_input1(val2)





In [ ]:
val2.shape

In [ ]:
prediction_2=model.predict(validation_2)

In [ ]:
def fill_contour(image):
  contours, hierarchy = cv2.findContours(image, 4, 1)
  points = np.concatenate((contours),)
  points = contours[0]
  for i in contours:
    if(len(i) > len(points)): points = i

  points = points + 1

  filled_segmentation = np.zeros((image.shape[0], image.shape[1],3))
  filled_segmentation = cv2.fillPoly(filled_segmentation, [np.int32(points)], [255,255,255])

  cv2_imshow(filled_segmentation)


In [ ]:
for index,img in enumerate(prediction_2):
  img=(img>=0.5)
  img = img.astype(int)
  img*=255
  cv2_imshow(val2[index])
  # cv2_imshow(img)
  fill_contour(img)
  